In [12]:
from torch_geometric.loader import DataLoader
import torch
from torch_geometric.nn import GNNExplainer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [13]:
torch.__version__

'1.11.0'

In [14]:
device = torch.device(2)
# device = torch.device("cpu")
device

device(type='cuda', index=2)

In [15]:
from utils_data import TestbedDataset
from models import GATNet_E, GATNet, GCNNet, RGCNNet

In [16]:
model = RGCNNet()
model_path = 'root_folder/root_026/models/model_RGCN-EP300-SW801010_GDSC.model'

# model = GCNNet()
# model_path = 'root_folder/root_020/models/model_GCN-EP300-SW801010_GDSC.model'
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)

RGCNNet(
  (conv1): RGCNConv(334, 334, num_relations=4)
  (conv2): RGCNConv(334, 668, num_relations=4)
  (conv3): RGCNConv(668, 1336, num_relations=4)
  (fc_g1): Linear(in_features=1336, out_features=1024, bias=True)
  (fc_g2): Linear(in_features=1024, out_features=128, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (conv_xt_1): Conv1d(1, 32, kernel_size=(8,), stride=(1,))
  (pool_xt_1): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv_xt_2): Conv1d(32, 64, kernel_size=(8,), stride=(1,))
  (pool_xt_2): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv_xt_3): Conv1d(64, 128, kernel_size=(8,), stride=(1,))
  (pool_xt_3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (fc1_xt): Linear(in_features=4224, out_features=128, bias=True)
  (fc1): Linear(in_features=256, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (out): Line

In [17]:
branch_folder = "root_folder/root_026"
# branch_folder = "root_folder/root_020"

dataset = 'GDSC'
test_data = TestbedDataset(root=branch_folder, dataset=dataset+'_test_mix')

Pre-processed data found: root_folder/root_026/processed/GDSC_test_mix.pt, loading ...


In [18]:
test_batch = 1
test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)

In [19]:
i = 0
for data in test_loader:
    i += 1
    print(data)
    if i >= 5:
        break

DataBatch(x=[37, 334], edge_index=[2, 82], y=[1], edge_features=[82], smiles=[1], drug_name=[1], cell_line_name=[1], target=[1, 1, 1000], c_size=[1], batch=[37], ptr=[2])
DataBatch(x=[24, 334], edge_index=[2, 54], y=[1], edge_features=[54], smiles=[1], drug_name=[1], cell_line_name=[1], target=[1, 1, 1000], c_size=[1], batch=[24], ptr=[2])
DataBatch(x=[43, 334], edge_index=[2, 96], y=[1], edge_features=[96], smiles=[1], drug_name=[1], cell_line_name=[1], target=[1, 1, 1000], c_size=[1], batch=[43], ptr=[2])
DataBatch(x=[17, 334], edge_index=[2, 34], y=[1], edge_features=[34], smiles=[1], drug_name=[1], cell_line_name=[1], target=[1, 1, 1000], c_size=[1], batch=[17], ptr=[2])
DataBatch(x=[31, 334], edge_index=[2, 72], y=[1], edge_features=[72], smiles=[1], drug_name=[1], cell_line_name=[1], target=[1, 1, 1000], c_size=[1], batch=[31], ptr=[2])


In [20]:
# test one drug
one = next(iter(test_loader))
data = one.to(device)
print(data)

DataBatch(x=[37, 334], edge_index=[2, 82], y=[1], edge_features=[82], smiles=[1], drug_name=[1], cell_line_name=[1], target=[1, 1, 1000], c_size=[1], batch=[37], ptr=[2])


In [21]:
data.edge_index

tensor([[ 0,  1,  1,  2,  2,  2,  3,  3,  4,  4,  5,  5,  5,  6,  6,  7,  7,  8,
          8,  8,  9,  9, 10, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 15,
         16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 21, 21, 21, 22, 22,
         22, 23, 23, 24, 24, 25, 25, 26, 27, 27, 28, 28, 28, 29, 29, 30, 30, 30,
         31, 31, 31, 32, 32, 33, 33, 34, 35, 36],
        [ 1,  0,  2,  1,  3,  7,  2,  4,  3,  5,  4,  6,  8,  5,  7,  6,  2,  5,
          9, 12,  8, 10,  9, 11, 13, 10, 12, 11,  8, 10, 14, 13, 15, 14, 16, 20,
         15, 17, 16, 18, 36, 17, 19, 24, 18, 20, 21, 19, 15, 19, 22, 27, 21, 23,
         25, 22, 24, 23, 18, 22, 26, 25, 21, 28, 27, 29, 33, 28, 30, 29, 31, 35,
         30, 32, 34, 31, 33, 32, 28, 31, 30, 17]], device='cuda:2')

In [22]:
explainer = GNNExplainer(model, lr=1e-4, epochs=300, return_type='regression')

x = data.x
edge_index = data.edge_index
y = data.y
batch_drug = data.batch
edge_features = data.edge_features
x_cell_mut = data.target

# model_args = (
#     x_cell_mut,
#     batch_drug,
#     edge_features
# )

kwargs = {
    "x_cell_mut": x_cell_mut,
    "batch_drug": batch_drug,
    "edge_feat": edge_features
}

node_feature_mask, edge_mask = explainer.explain_graph(x = x, edge_index = edge_index, x_cell_mut = x_cell_mut, edge_feat = edge_features)
# node_feature_mask, edge_mask = explainer.explain_graph(x, edge_index, **kwargs)



Explain graph:   0%|                                                                                                                                                              | 0/300 [00:00<?, ?it/s]

debug h torch.Size([37, 334])
debug edge torch.Size([2, 82])
36 82
36 119


AssertionError: 

In [23]:
edge_features[edge_features==0].shape

torch.Size([36])

In [24]:
x.shape

torch.Size([37, 334])

In [25]:
edge_index.size(1)

82

In [ ]:
# debug rgcn model
(N, F) = x.size()
std = 0.1
node_feat_mask = torch.nn.Parameter(torch.randn(1, F) * std)
node_feat_mask

In [ ]:
h = x * node_feat_mask.sigmoid().cuda()
h

In [ ]:
h.shape

- node_feature_mask: size = node feature length (334/...), importance of each node feature (invariant of atoms)
- edge_mask: size = edge number, importance of edges, stored in the sequence of edge_index (0,1),(0,5),(1,...

In [ ]:
one.edge_index

In [ ]:
edge_mask

In [ ]:
node_feature_mask

In [ ]:
edge_mask_cpu = edge_mask.to('cpu')
edge_index_cpu = edge_index.to('cpu')

In [ ]:
ax, G = explainer.visualize_subgraph(node_idx=None, edge_index=edge_index_cpu, edge_mask=edge_mask_cpu)
plt.show()

In [ ]:
G[0]

In [ ]:
G[5][0]

In [ ]:
explainer = GNNExplainer(model, lr=1e-4, epochs=300)

for data in test_loader:
    x = data.x
    edge_index = data.edge_index
    y = data.y
    batch = data.batch
    edge_features = data.edge_features
    target = data.target
    
    node_feature_mask, edge_mask = explainer.explain_graph(x, edge_index, batch=batch, edge_feat = edge_features)
    ax, G = explainer.visualize_subgraph(edge_index, edge_mask, y=y)